In [1]:
# --------------------------------------------
#                   TEST LCU
# --------------------------------------------
from src import lcu
lcu = lcu.LCU()

instr_list = ["NOP", "EXIT", "SADD R1, ZERO, LAST", "SADD R1, SRF(3), LAST", "SADDI R1, ONE, 7", "SSUB SRF(4), SRF(4), SRF(4)", "JUMP ONE, 7", "BGEPD ONE, ZERO, 5"]

for instr in instr_list:
    srf_read, srf_str, word = lcu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
EXIT: -1, -1, 0x1c00
SADD R1, ZERO, LAST: -1, -1, 0xb8340
SADD R1, SRF(3), LAST: 3, -1, 0xb0340
SADDI R1, ONE, 7: -1, -1, 0xfc347
SSUB SRF(4), SRF(4), SRF(4): 4, 4, 0x90400
JUMP ONE, 7: -1, -1, 0xfda07
BGEPD ONE, ZERO, 5: -1, -1, 0xdd605


In [2]:
# --------------------------------------------
#                   TEST LSU
# --------------------------------------------
from src import lsu
lsu = lsu.LSU()

instr_list = ["SADD R0, ONE, ONE/LD.VWR VWR_A", "SADD R0, SRF(5), ONE/SH.IL.UP", "SADD SRF(5), SRF(5), ONE/LD.VWR SRF"]

for instr in instr_list:
    srf_read, srf_str, word = lsu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

SADD R0, ONE, ONE/LD.VWR VWR_A: -1, -1, 0x45538
SADD R0, SRF(5), ONE/SH.IL.UP: 5, -1, 0xc4538
SADD SRF(5), SRF(5), ONE/LD.VWR SRF: 5, 5, 0x5c530


In [3]:
# --------------------------------------------
#                   TEST RCs
# --------------------------------------------
from src import rc
rc = rc.RC()

instr_list = ["NOP", "SADD VWR_A, VWR_A, VWR_B", "SADD VWR_A, SRF(3), VWR_B", "LOR R0, RCB, MIN_INT", "MUL.FP R0, RCB, MIN_INT"]
# DIV.FP R0, RCB, MIN_INT") Error: float point div not supported
for instr in instr_list:
    srf_read, srf_str, vwr_str, word = rc.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
SADD VWR_A, VWR_A, VWR_B: -1, -1, 0x420
SADD VWR_A, SRF(3), VWR_B: 3, -1, 0xc420
LOR R0, RCB, MIN_INT: -1, -1, 0x1f522
MUL.FP R0, RCB, MIN_INT: -1, -1, 0x1f5a2


In [4]:
# --------------------------------------------
#                   TEST MXCU
# --------------------------------------------
from src import mxcu
mxcu = mxcu.MXCU()

instr_list = ["NOP", "SADD R1, ONE, LAST", "LOR R1, ONE, SRF(3)"]
srf_sel = -1
srf_we = 0
alu_srf_write = 0
vwr_row_we = [0,0,0,0]
vwr_sel = 0
for instr in instr_list:
    word = mxcu.asmToHex(instr, srf_sel, srf_we, alu_srf_write, vwr_row_we, vwr_sel)
    print(instr + ": " + word.get_word_in_hex())

NOP: 0x0
SADD R1, ONE, LAST: 0x5699000
LOR R1, ONE, SRF(3): 0x54690c0


In [5]:
# --------------------------------------------
#                   COMPILE
# --------------------------------------------
from src import simulator
kernel_path='./kernels/test_asm/'
sim = simulator.SIMULATOR()
sim.compileAsm(kernel_path, version="_v1", nInstrPerCol=11, colUsage=[True, False], imem_start_addr=3)

Processing file: ./kernels/test_asm/instructions_asm_v1.csv...


In [6]:
from src import simulator
kernel_path='./kernels/test_asm/'
sim = simulator.SIMULATOR()
# Using two cols
sim.compileAsm(kernel_path, version="_v2", nInstrPerCol=5, colUsage=[True, True], imem_start_addr=3)

Processing file: ./kernels/test_asm/instructions_asm_v2.csv...


In [7]:
# --------------------------------------------
#                   SIMULATE
# --------------------------------------------
from src.simulator import *
from random import randint

sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/test_asm/' # Path
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 11 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v1"

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector by vector in the positions you want
random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
nline = 3
sim.setSPMLine(nline, random_vector)
# Or, load the entire content of the SPM
data = []
for nline in range(SPM_NLINES):
    random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
    data.append(random_vector)
sim.loadSPMData(data)

# --------------------------------------------
#              COMPILE ASM TO HEX
# --------------------------------------------
# If hexadecimal is not provided, here we generate it
sim.compileAsm(kernel_path, version=version, nInstrPerCol=nInstrPerCol, colUsage=column_usage, imem_start_addr=imem_add_start)

# --------------------------------------------
#                 LOAD KERNEL
# --------------------------------------------

# This needs the hex instructions, if you don't provide them, generate then compiling the asm
sim.kernel_load(kernel_path, version=version, kernel_number=kernel_number)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = []
show_srf = []
show_lsu = []
show_rcs = [[],[],[],[]]
show_mxcu = []
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

sim.displaySPMLine(3)

sim.run(kernel_number, display_ops=display_ops)

Processing file: ./kernels/test_asm/instructions_asm_v1.csv...
Processing file: ./kernels/test_asm/instructions_hex_v1.csv...
SPM 3: [80, 99, 35, 57, 90, 96, 41, 42, 20, 6, 15, 5, 62, 14, 29, 39, 86, 63, 7, 26, 12, 78, 6, 69, 17, 19, 10, 45, 67, 100, 77, 54, 30, 97, 42, 94, 97, 50, 84, 42, 98, 3, 39, 73, 29, 43, 69, 68, 38, 85, 70, 54, 56, 82, 12, 55, 29, 79, 39, 78, 95, 27, 90, 1, 1, 90, 43, 85, 99, 34, 95, 12, 7, 34, 72, 66, 94, 20, 8, 34, 77, 18, 88, 11, 82, 71, 16, 23, 3, 94, 71, 61, 8, 93, 82, 1, 1, 69, 69, 15, 54, 28, 73, 69, 52, 44, 68, 20, 89, 22, 57, 65, 85, 6, 16, 80, 47, 21, 19, 65, 100, 57, 98, 69, 10, 33, 20, 8, ]
PC: 0
Col: 0
LSU: 0x45538
MXCU: 0x5699000
RC: 0x0
RC: 0x0
RC: 0x0
RC: 0x0
LCU: 0xdc340
PC: 1
Col: 0
LSU: 0xc4538
MXCU: 0x569914f
RC: 0x420
RC: 0x420
RC: 0x420
RC: 0x420
LCU: 0xe8547
PC: 2
Col: 0
LSU: 0x5c530
MXCU: 0xecf
RC: 0xc420
RC: 0xc420
RC: 0xc420
RC: 0xc420
LCU: 0x0
PC: 3
Col: 0
LSU: 0xc4538
MXCU: 0x5469140
RC: 0x1f522
RC: 0x1f522
RC: 0x1f522
RC: 0x1f522
LC